## Invoice-Project: brands 
-EDA -cleaning -heuristic baseline  

Date: April 3rd, 2022 

Author: Nana 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
maindir = '/content/drive/MyDrive/FinTech-final-project'
import pandas as pd
filepath = f'{maindir}/東吳課程_發票資料集/品牌資料集/brand_train.csv'
branddf = pd.read_csv(filepath)

## Text normalization

In [3]:
brand_map = branddf['brand'].value_counts().to_dict()

In [4]:
brand_map.items()

dict_items([('耐吉 NIKE', 770), ('adidas', 724), ('MAYBELLINE 媚比琳', 635), ('羅技 Logitech', 611), ('MENTHOLATUM 曼秀雷敦', 549), ('義美', 536), ('KATE TOKYO 凱婷', 505), ('蘋果 Apple', 489), ('高露潔', 488), ('media 媚點', 482), ('SENKA 專科', 437), ('KIRIN', 433), ('台啤', 397), ('1028', 386), ('國際牌 Panasonic', 385), ('Za', 373), ('NEW BALANCE', 364), ('essence cosmetics 艾森絲', 351), ('黑人牙膏', 328), ('七星', 328), ('澎澎', 313), ('三星 SAMSUNG', 301), ('森田藥粧', 286), ('Cellina 雪芙蘭', 281), ('康乃馨', 274), ('INTEGRATE 櫻特芮', 274), ('NIVEA 妮維雅', 267), ('農心', 265), ('蜜妮Biore', 263), ('M.A.C', 261), ('舒酸定', 260), ('SKECHERS', 255), ('白蘭氏', 251), ('NAILHOLIC', 246), ('Catrice 卡翠絲', 243), ('刷樂', 242), ('Biore 蜜妮', 241), ('熊寶貝', 231), ('獅王', 230), ('舒潔', 230), ('一匙靈', 227), ('寶僑', 225), ('CHANEL 香奈兒', 225), ('黑松 HeySong', 223), ('Bioré', 223), ('可口可樂 Coca Cola', 220), ('桂格 QUAKER', 220), ('LION 獅王', 218), ('KISSME 奇士美', 217), ('Dashing Diva', 214), ("KIEHL'S 契爾氏", 213), ('特福 Tefal', 208), ('原萃', 205), ('DR.WU 達爾膚', 204), ('UND

In [5]:
# brand name lexicalizations dictionary
brand_invlexdict = {}
brand_lexdict = {}
for complete_name in brand_map.keys():
  lexics = complete_name.split(' ')
  brand_lexdict[complete_name] = lexics
  brand_invlexdict[complete_name] = complete_name
  for lex in lexics:
    if len(lex.replace(' ', '')) == 0:
      continue
    if lex not in brand_invlexdict:
      brand_invlexdict[lex] = complete_name
      continue
    # 完整的名字通常越長
    if len(brand_invlexdict[lex]) < len(complete_name):
      brand_invlexdict[lex] = complete_name
    
# inverted dictionary 
print(brand_invlexdict['慧心'])
print(brand_invlexdict['慧心貓糧 SmartHeart'])
print(brand_invlexdict['SmartHeart'])
# Question: 在train dataset中同時有'慧心 SmartHeart'與'慧心貓糧 SmartHeart'的標記，但感覺是同一家公司。
# 那麼意思是說，在test set上，如果遇到該品牌，標'慧心 SmartHeart'或'慧心貓糧 SmartHeart'都會被判斷為正確嗎？
# 還是說只有其中一個是對的？如果只有其中一個是對的，我們要怎麼判斷是哪個呢？有正確的品牌標記的清單可循嗎？

慧心 SmartHeart
慧心貓糧 SmartHeart
慧心貓糧 SmartHeart


In [6]:
import os 
os.makedirs(f'{maindir}/data', exist_ok = True)
filep = f'{maindir}/data/brand_counts.json'
branddf['brand'].value_counts().to_json(filep, 
                                        force_ascii = False, 
                                        indent = 4)

## quantiles/percentiles

In [ ]:
import numpy as np
vals = list(brand_map.values())
percs = ['中位數','3/4位數', 'pr 95', 'pr 99']
ints = [50, 75, 95, 99]
for x, ith in zip(percs, ints):
  print(f'{x}: {np.percentile(vals, ith)}')

中位數: 3.0
3/4位數: 11.0
pr 95: 82.0
pr 99: 225.65999999999985


In [7]:
import re
def clean_puncts(x):
  '''
  match English, numbers, chinese;
  clean up all spaces
  '''
  x = str(x)
  repat = '[\u4e00-\u9fff0-9a-zA-Z]+'
  res = re.findall(repat, x)
  return ''.join(res)
def get_brand_(x):
  x = str(x)
  repat = '\【(.*?)\】'
  first_match = re.match(repat, x)
  # print('first match:', first_match)
  if first_match:
    return first_match.group(0)[1:-1] 
  # 有點沒效率
  for k in brand_invlexdict.keys():
    if k in x:
      # print(f'key:{k}')
      return brand_invlexdict[k]
  return x[:3]

text = 'S11益生菌(2g*30包/盒)'	
text = '【龍宏】辣豆瓣醬460g'
text = '澤井 MugMug掛耳咖啡10P-濃郁(130g)'
# text = '龜丸大豆本釀醬油500m'
print(clean_puncts(text)) 
print(get_brand_(text))    # 應該要是S11益生菌，這是machine-annotated?

澤井MugMug掛耳咖啡10P濃郁130g
澤井


In [8]:
branddf['clean_name'] = branddf.apply(lambda x: clean_puncts(x['name']), axis = 1)
branddf['heuristic_brand'] =  branddf.apply(lambda x: get_brand_(x['name']), axis = 1)

In [ ]:
branddf

,name,occurred_at,iv_price,brand,units,unit_price,total_price,channel,clean_name,heuristic_brand
0,元山熱水瓶YS-540AP,2021-02-07,1650,元山,1.0,1490.0,1188.0,738fbc9320a9106d8905bef71a02bb99,元山熱水瓶YS540AP,元山
1,356612@姍拉娜治痘洗面乳150G,2021-02-02,1053,姍拉娜,1.0,85.0,85.0,d10c78673e1d88714ba645fadc3de0d1,356612姍拉娜治痘洗面乳150G,姍拉娜
2,台鹽海洋鹼性離子水 600ML,2021-02-04,609,台鹽,1.0,19.0,19.0,d7769307282f69448322f9306bdf9700,台鹽海洋鹼性離子水600ML,台鹽
3,妮傲絲翠果酸深層保養乳液,2021-02-14,1142,NEO-TEC妮傲絲翠,1.0,1200.0,1200.0,7a0a2154662bc9038594f04c886c87d8,妮傲絲翠果酸深層保養乳液,NEO-TEC 妮傲絲翠
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,2021-02-01,198,舒潔,1.0,198.0,198.0,ab7febfe145bba3da1c0fb19507f1703,舒潔棉柔舒適迪士尼抽取式衛生紙100抽16入,舒潔
...,...,...,...,...,...,...,...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8TWGD6622HW,2021-02-07,35291,惠而浦 Whirlpool,1.0,35291.0,35291.0,4a3d67cc0cd76bf56f18b40ed671bb05,惠而浦16公斤瓦斯型滾筒蒸氣可堆疊乾衣機8TWGD6622HW,惠而浦 whirlpool
70042,康乃馨成人紙尿褲L,2021-02-04,1041,康乃馨,1.0,194.0,194.0,9742158c78b35d77c6351ecbfb60897b,康乃馨成人紙尿褲L,康乃馨
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,2021-02-04,299,芮芙茹 Reveur,1.0,299.0,299.0,0f20279a949fd072f0e3858f97d0654a,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur
70044,Crest長效鎖白牙膏-輕柔鑽白99g,2021-02-13,401,crest,1.0,99.0,99.0,6757b9193b3446acb4b97a05649b3c7e,Crest長效鎖白牙膏輕柔鑽白99g,Peach C


In [ ]:
branddf.to_csv(f'{maindir}/data/preproc_brand_train.csv')

In [ ]:
# brands = branddf.groupby('brand')  # brands.first()
import json
fp1 = f'{maindir}/data/brand_lexics.json'
fp2 = f'{maindir}/data/inverted_brand_lexics.json'
with open (fp1, 'w') as f:
  json.dump(brand_lexdict, f, indent = 4, ensure_ascii = False)
with open (fp2, 'w') as f:
  json.dump(brand_invlexdict, f, indent = 4, ensure_ascii = False)

## Profiler Report
用的是原始資料集，
但因為原始資料集有標記品牌多種表達（multiple lexicalizations）的問題，
數據可能有所失真。

In [ ]:
!pip -q install pandas-profiling==2.8.0

In [ ]:
# !pip uninstall pandas-profiling

Found existing installation: pandas-profiling 2.8.0
Uninstalling pandas-profiling-2.8.0:
  Would remove:
    /usr/local/bin/pandas_profiling
    /usr/local/lib/python3.7/dist-packages/pandas_profiling-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas_profiling/*
Proceed (y/n)? y
  Successfully uninstalled pandas-profiling-2.8.0


In [ ]:
from pandas_profiling import ProfileReport
file_prefix= 'brand_train'
filepath = f'{maindir}/東吳課程_發票資料集/品牌資料集/brand_train.csv'
df= pd.read_csv(filepath)
df.fillna('', inplace = True) 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title = file_prefix, explorative = True)
profile.to_file(f'{maindir}/data/{file_prefix}_EDA.html') # may need to fix version issues 

Summarize dataset:   0%|          | 0/22 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Heuristic Baseline:
### Macro f1
reflections: need to get「正確」的品牌名


In [10]:
import numpy as np
from sklearn.metrics import f1_score

In [11]:
y_true = branddf['brand']
y_pred = branddf['heuristic_brand']
f1_score(y_true, y_pred, average = 'macro') 

0.19162285928911968

### Jaccard

In [12]:
!pip -q install textdistance

In [13]:
from textdistance import jaccard
running_score = 0 
for t, p in zip(y_true, y_pred):
  running_score += jaccard(t, p)
running_score/(len(y_true))

0.6505755685570801